In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import polars as pl
%reload_ext autoreload
%autoreload 2

In [3]:
df = pd.read_csv(
    "data/Euronext_Equities_2025-08-19.csv",
    sep=";",          # Euronext uses semicolons
    skiprows=0,       # adjust if file has metadata rows
    on_bad_lines="skip"  # skip problematic rows if any
)


In [4]:
possible_markets = ['Oslo Børs', 'Euronext Paris', 'Euronext Amsterdam', 'Euronext Brussels', 'Euronext Lisbon']
filtered_df = df[df['Market'].isin(possible_markets)]
filtered_df

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,"133,50","135,00","133,00","133,10",19/08/2025 16:25,CET,107862,"14447074,90","133,10",19/08/2025
14,74SOFTWARE,FR0011040500,74SW,Euronext Paris,EUR,"39,10","40,00","39,10","39,80",19/08/2025 17:35,CET,3538,"139815,50","39,80",19/08/2025
20,AALBERTS NV,NL0000852564,AALB,Euronext Amsterdam,EUR,"30,12","30,80","30,12","30,74",19/08/2025 17:35,CET,159908,"4903828,56","30,74",19/08/2025
24,AB INBEV,BE0974293251,ABI,Euronext Brussels,EUR,"53,20","53,58","52,90","53,42",19/08/2025 17:39,CET,954025,"50925650,92","53,42",19/08/2025
25,AB SCIENCE,FR0010557264,AB,Euronext Paris,EUR,"1,33","1,344","1,32","1,32",19/08/2025 17:35,CET,110739,"147289,34","1,32",19/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4024,ZALARIS,NO0010708910,ZAL,Oslo Børs,NOK,"76,00","77,00","76,00","77,00",19/08/2025 16:25,CET,882,"67622,40","77,00",19/08/2025
4025,ZAPTEC,NO0010713936,ZAP,Oslo Børs,NOK,"26,25","26,50","25,55","25,95",19/08/2025 16:27,CET,554446,"14464193,45","25,95",19/08/2025
4027,ZCI LIMITED,BMG9887P1068,CV,Euronext Paris,EUR,"0,17","0,20","0,17","0,20",04/08/2015 16:20,CET,21826,"3755,94","0,20",19/08/2025
4029,ZELLUNA,NO0013524942,ZLNA,Oslo Børs,NOK,"13,50","13,676","13,372","13,502",19/08/2025 16:25,CET,4987,"67291,022","13,502",19/08/2025


In [5]:
market_grouped_equities = filtered_df.groupby('Market')

In [6]:
market_grouped_equities.get_group('Oslo Børs')

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,"133,50","135,00","133,00","133,10",19/08/2025 16:25,CET,107862,"14447074,90","133,10",19/08/2025
39,ABG SUNDAL COLLIER,NO0003021909,ABG,Oslo Børs,NOK,"6,96","7,05","6,95","6,95",19/08/2025 16:25,CET,2305357,"17000246,86","6,95",19/08/2025
44,ABL GROUP,NO0010715394,ABL,Oslo Børs,NOK,"9,36","9,36","9,14","9,14",19/08/2025 16:15,CET,47463,"437236,10","9,14",19/08/2025
116,AF GRUPPEN,NO0003078107,AFG,Oslo Børs,NOK,"160,20","163,60","160,00","163,40",19/08/2025 16:25,CET,29299,"4764223,60","163,40",19/08/2025
134,AGILYX,NO0010872468,AGLX,Oslo Børs,NOK,"24,50","25,00","24,50","25,00",19/08/2025 16:25,CET,10652,"264156,40","25,00",19/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,WILH. WILHELMSEN B,NO0010576010,WWIB,Oslo Børs,NOK,"470,00","475,00","470,00","470,50",19/08/2025 16:27,CET,6414,"3026836,00","470,50",19/08/2025
4016,YARA INTERNATIONAL,NO0010208051,YAR,Oslo Børs,NOK,"376,60","378,80","374,50","375,60",19/08/2025 16:26,CET,495690,"186290912,288","375,60",19/08/2025
4024,ZALARIS,NO0010708910,ZAL,Oslo Børs,NOK,"76,00","77,00","76,00","77,00",19/08/2025 16:25,CET,882,"67622,40","77,00",19/08/2025
4025,ZAPTEC,NO0010713936,ZAP,Oslo Børs,NOK,"26,25","26,50","25,55","25,95",19/08/2025 16:27,CET,554446,"14464193,45","25,95",19/08/2025


In [7]:
df = df.dropna()
oslo_stocks = df.loc[df['Market'] == 'Oslo Børs']

In [8]:
oslo_stocks

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,"133,50","135,00","133,00","133,10",19/08/2025 16:25,CET,107862,"14447074,90","133,10",19/08/2025
39,ABG SUNDAL COLLIER,NO0003021909,ABG,Oslo Børs,NOK,"6,96","7,05","6,95","6,95",19/08/2025 16:25,CET,2305357,"17000246,86","6,95",19/08/2025
44,ABL GROUP,NO0010715394,ABL,Oslo Børs,NOK,"9,36","9,36","9,14","9,14",19/08/2025 16:15,CET,47463,"437236,10","9,14",19/08/2025
116,AF GRUPPEN,NO0003078107,AFG,Oslo Børs,NOK,"160,20","163,60","160,00","163,40",19/08/2025 16:25,CET,29299,"4764223,60","163,40",19/08/2025
134,AGILYX,NO0010872468,AGLX,Oslo Børs,NOK,"24,50","25,00","24,50","25,00",19/08/2025 16:25,CET,10652,"264156,40","25,00",19/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,WILH. WILHELMSEN B,NO0010576010,WWIB,Oslo Børs,NOK,"470,00","475,00","470,00","470,50",19/08/2025 16:27,CET,6414,"3026836,00","470,50",19/08/2025
4016,YARA INTERNATIONAL,NO0010208051,YAR,Oslo Børs,NOK,"376,60","378,80","374,50","375,60",19/08/2025 16:26,CET,495690,"186290912,288","375,60",19/08/2025
4024,ZALARIS,NO0010708910,ZAL,Oslo Børs,NOK,"76,00","77,00","76,00","77,00",19/08/2025 16:25,CET,882,"67622,40","77,00",19/08/2025
4025,ZAPTEC,NO0010713936,ZAP,Oslo Børs,NOK,"26,25","26,50","25,55","25,95",19/08/2025 16:27,CET,554446,"14464193,45","25,95",19/08/2025


In [9]:
oslo_stocks
oslo_tickers = list(oslo_stocks['Symbol'].values)
oslo_tickers = ['OSEBX.OL'] + ['.'.join([i, 'OL']) for i in oslo_tickers]
ticker_data = []

In [10]:
oslo_tickers

['OSEBX.OL',
 '2020.OL',
 'ABG.OL',
 'ABL.OL',
 'AFG.OL',
 'AGLX.OL',
 'AIRX.OL',
 'AKAST.OL',
 'AKER.OL',
 'AKBM.OL',
 'AKRBP.OL',
 'ACC.OL',
 'AKH.OL',
 'AKSO.OL',
 'AKVA.OL',
 'AMSC.OL',
 'ARCH.OL',
 'AZT.OL',
 'AFK.OL',
 'ARR.OL',
 'ATEA.OL',
 'ASAS.OL',
 'ASA.OL',
 'AURG.OL',
 'AUSS.OL',
 'AUTO.OL',
 'AGAS.OL',
 'ACR.OL',
 'B2I.OL',
 'BAKKA.OL',
 'BGBIO.OL',
 'BEWI.OL',
 'BIEN.OL',
 'BNOR.OL',
 'BONHR.OL',
 'BOR.OL',
 'BRG.OL',
 'BOUV.OL',
 'BWE.OL',
 'BWLPG.OL',
 'BWO.OL',
 'BMA.OL',
 'CADLR.OL',
 'CAPSL.OL',
 'CAVEN.OL',
 'CRNA.OL',
 'CLOUD.OL',
 'CMBTO.OL',
 'CONTX.OL',
 'DNB.OL',
 'DNO.OL',
 'DOFG.OL',
 'EIOF.OL',
 'EMGS.OL',
 'ELK.OL',
 'ELABS.OL',
 'ELMRA.OL',
 'ELO.OL',
 'ENDUR.OL',
 'ENSU.OL',
 'ENTRA.OL',
 'ENVIP.OL',
 'EQNR.OL',
 'EQVA.OL',
 'EPR.OL',
 'FLNG.OL',
 'FRO.OL',
 'GENT.OL',
 'GJF.OL',
 'GOGL.OL',
 'GOD.OL',
 'GSF.OL',
 'GYL.OL',
 'HAFNI.OL',
 'HGSB.OL',
 'HAVI.OL',
 'HERMA.OL',
 'HEX.OL',
 'HPUR.OL',
 'HSHP.OL',
 'HBC.OL',
 'HYPRO.OL',
 'HAUTO.OL',
 'HSPG.OL'

In [26]:
j = yf.Ticker(oslo_tickers[42])

In [34]:
j.income_stmt

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,-4.978800e+04,0.000000e+00,0.000000e+00,0.000000e+00,NaN
Tax Rate For Calcs,3.600000e-02,0.000000e+00,0.000000e+00,0.000000e+00,NaN
Normalized EBITDA,1.282510e+08,4.617400e+07,6.380100e+07,2.725500e+07,NaN
Total Unusual Items,-1.383000e+06,-8.752000e+06,-4.047000e+06,1.030000e+05,NaN
Total Unusual Items Excluding Goodwill,-1.383000e+06,-8.752000e+06,-4.047000e+06,1.030000e+05,NaN
Net Income From Continuing Operation Net Minority Interest,6.506900e+07,1.149800e+07,3.554100e+07,7.451000e+06,NaN
Reconciled Depreciation,5.659500e+07,2.304800e+07,2.268400e+07,1.647900e+07,NaN
Reconciled Cost Of Revenue,7.015500e+07,3.734400e+07,2.787300e+07,2.281400e+07,NaN
EBITDA,1.268680e+08,3.742200e+07,5.975400e+07,2.735800e+07,NaN
EBIT,7.027300e+07,1.437400e+07,3.707000e+07,1.087900e+07,NaN


In [35]:
j.balance_sheet

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,93538.0,0.0,NaN,NaN,NaN
Ordinary Shares Number,350864045.0,311409868.0,197600000.0,138574468.0,NaN
Share Issued,350957583.0,311409868.0,197600000.0,138574468.0,NaN
Net Debt,512553000.0,108964000.0,95990000.0,70767000.0,NaN
Total Debt,581988000.0,206565000.0,115281000.0,73582000.0,NaN
...,...,...,...,...,...
Loans Receivable,NaN,NaN,NaN,NaN,6000.0
Accounts Receivable,62772000.0,29960000.0,18235000.0,18424000.0,NaN
Cash Cash Equivalents And Short Term Investments,58464000.0,96608000.0,19012000.0,2308000.0,NaN
Cash And Cash Equivalents,58464000.0,96608000.0,19012000.0,2308000.0,NaN


In [36]:
j.cash_flow

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Free Cash Flow,-522851000.0,-3547000.0,-196372000.0,-133175000.0,NaN
Repurchase Of Capital Stock,-1283000.0,0.0,0.0,NaN,NaN
Repayment Of Debt,-12591000.0,-115569000.0,-90293000.0,-10285000.0,NaN
Issuance Of Debt,365975000.0,199935000.0,129526000.0,8998000.0,NaN
Issuance Of Capital Stock,154956000.0,0.0,183250000.0,79218000.0,NaN
...,...,...,...,...,...
Depreciation,NaN,22837000.0,22684000.0,16479000.0,15482000.0
Operating Gains Losses,-244000.0,766000.0,NaN,NaN,NaN
Gain Loss On Investment Securities,-427000.0,766000.0,NaN,NaN,NaN
Gain Loss On Sale Of PPE,183000.0,0.0,0.0,NaN,NaN


In [38]:
j.info.get('trailingPE', None)

18.161291

In [50]:
'quickRatio' in list(j.info.keys())

True